In [0]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

In [0]:
CLIENT_ID = '2VJSZMEZBVCII3HYWBCNGEDFTEI3JAVBB3BU0X1XRGN4LGM1' 
CLIENT_SECRET = '0JYZ0FESUQ02YKG3WUKAAOIHTIMVKHM1TYXYBW1ML3PF4QGB'
VERSION = '20200417'
LIMIT= 50



In [65]:
city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [70]:
search_query = 'Hotel'
radius = 500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

result_hotel = requests.get(url).json()

result_hotel

venue_hotel = result_hotel['response']['venues']

df = json_normalize(venue_hotel)
df.head()



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4ab2d511f964a5209b6c20e3,Sheraton Centre Toronto Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1587139133,False,123 Queen Street West,at York St.,43.650594,-79.384530,"[{'label': 'display', 'lat': 43.6505944, 'lng'...",324,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,NaN
1,51d212c3498ebf27dc469bc9,Chelsea Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1587139133,False,33 Gerrard Street West,at Yonge St,43.658498,-79.383097,"[{'label': 'display', 'lat': 43.65849759157591...",562,M5G 1Z4,CA,Toronto,ON,Canada,"[33 Gerrard Street West (at Yonge St), Toronto...",NaN,NaN
2,4af96fbbf964a520c01122e3,One King West Hotel & Residence,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1587139133,False,1 King St W,at Yonge St.,43.649139,-79.377876,"[{'label': 'display', 'lat': 43.6491395, 'lng'...",686,M5H 1A1,CA,Toronto,ON,Canada,"[1 King St W (at Yonge St.), Toronto ON M5H 1A...",NaN,NaN
3,4ae61cf6f964a520caa421e3,Pantages Hotel & Spa,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1587139133,False,200 Victoria St,at Shuter St,43.654498,-79.379035,"[{'label': 'display', 'lat': 43.65449797039222...",410,NaN,CA,Toronto,ON,Canada,"[200 Victoria St (at Shuter St), Toronto ON, C...",NaN,NaN
4,51e48697498eded9073c6c17,Crew Room Eaton Chelsea Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1587139133,False,33 Gerrard Street West,NaN,43.658094,-79.382711,"[{'label': 'display', 'lat': 43.65809435580944...",522,NaN,CA,Toronto,ON,Canada,"[33 Gerrard Street West, Toronto ON, Canada]",NaN,NaN


In [71]:
new_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')]+ ['id']
new_df = df.loc[:,new_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

new_df['categories'] = new_df.apply(get_category_type, axis=1)

new_df.columns = [column.split('.')[-1] for column in new_df.columns]

new_df.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,at York St.,43.650594,-79.384530,"[{'label': 'display', 'lat': 43.6505944, 'lng'...",324,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,4ab2d511f964a5209b6c20e3
1,Chelsea Hotel,Hotel,33 Gerrard Street West,at Yonge St,43.658498,-79.383097,"[{'label': 'display', 'lat': 43.65849759157591...",562,M5G 1Z4,CA,Toronto,ON,Canada,"[33 Gerrard Street West (at Yonge St), Toronto...",NaN,51d212c3498ebf27dc469bc9
2,One King West Hotel & Residence,Hotel,1 King St W,at Yonge St.,43.649139,-79.377876,"[{'label': 'display', 'lat': 43.6491395, 'lng'...",686,M5H 1A1,CA,Toronto,ON,Canada,"[1 King St W (at Yonge St.), Toronto ON M5H 1A...",NaN,4af96fbbf964a520c01122e3
3,Pantages Hotel & Spa,Hotel,200 Victoria St,at Shuter St,43.654498,-79.379035,"[{'label': 'display', 'lat': 43.65449797039222...",410,NaN,CA,Toronto,ON,Canada,"[200 Victoria St (at Shuter St), Toronto ON, C...",NaN,4ae61cf6f964a520caa421e3
4,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,NaN,43.658094,-79.382711,"[{'label': 'display', 'lat': 43.65809435580944...",522,NaN,CA,Toronto,ON,Canada,"[33 Gerrard Street West, Toronto ON, Canada]",NaN,51e48697498eded9073c6c17


In [72]:
new_df= new_df.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
new_df


,name,categories,address,lat,lng,postalCode,state,neighborhood
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON,NaN
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON,NaN
2,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON,NaN
3,Pantages Hotel & Spa,Hotel,200 Victoria St,43.654498,-79.379035,NaN,ON,NaN
4,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,43.658094,-79.382711,NaN,ON,NaN
5,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON,NaN
6,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON,NaN
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON,NaN
8,Sheraton Centre Hotel Club Lounge,Lounge,NaN,43.651063,-79.384527,M5H 2M9,ON,NaN
9,Sheraton Centre Toronto Hotel - Grand Ballroom,Ballroom,123 Queen Street West,43.651200,-79.384520,NaN,ON,Financial District


In [73]:
array= ['Hotel']
df_hotel= new_df.loc[new_df['categories'].isin(array)]
df_hotel

,name,categories,address,lat,lng,postalCode,state,neighborhood
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON,NaN
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON,NaN
2,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON,NaN
3,Pantages Hotel & Spa,Hotel,200 Victoria St,43.654498,-79.379035,NaN,ON,NaN
4,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,43.658094,-79.382711,NaN,ON,NaN
6,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON,NaN
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON,NaN
10,Stathcona Hotel,Hotel,NaN,43.654947,-79.386359,NaN,ON,NaN
15,650 Hotel,Hotel,650 Bay Street,43.657046,-79.384411,NaN,ON,NaN
16,Be SixFifty Hotel,Hotel,650 Bay Street,43.657120,-79.384560,NaN,ON,NaN


In [74]:
search_query = 'Park'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=2VJSZMEZBVCII3HYWBCNGEDFTEI3JAVBB3BU0X1XRGN4LGM1&client_secret=0JYZ0FESUQ02YKG3WUKAAOIHTIMVKHM1TYXYBW1ML3PF4QGB&ll=43.6534817,-79.3839347&v=20200417&query=Park&radius=10000&limit=50'

In [75]:
result_park = requests.get(url).json()
venue_park = result_park['response']['venues']

park_df = json_normalize(venue_park)
park_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4b170e47f964a5208cc123e3,Queen's Park Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1587139172,False,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",887,M5G 2P1,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",NaN
1,4b54deadf964a520a3d027e3,Grange Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1587139172,False,NaN,NaN,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.65248755299856...",663,NaN,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, To...",NaN
2,4b9177e4f964a520d2be33e3,TTC Streetcar #403 - Victoria park,"[{'id': '4bf58dd8d48988d1fc931735', 'name': 'L...",v-1587139172,False,NaN,NaN,43.650735,-79.386921,"[{'label': 'display', 'lat': 43.650735, 'lng':...",389,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN
3,4d3a1d23325ff04db2dd1845,Residences at College Park North,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",v-1587139172,False,763 Bay St.,btwn. College St. and Gerrard St.,43.659822,-79.385159,"[{'label': 'display', 'lat': 43.65982157793473...",712,M5G 2R3,CA,Toronto,ON,Canada,[763 Bay St. (btwn. College St. and Gerrard St...,NaN
4,4bfc5019da7120a132df49fd,St. Andrews Playground / Dog Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1587139172,False,450 Adelaide Street West,Brant St & Adelaide St W,43.647388,-79.398752,"[{'label': 'display', 'lat': 43.64738759338116...",1372,NaN,CA,Toronto,ON,Canada,[450 Adelaide Street West (Brant St & Adelaide...,NaN


In [76]:
new_column_park = ['name', 'categories'] + [col for col in park_df.columns if col.startswith('location.')]+ ['id']
new_df_park = park_df.loc[:,new_column_park]

def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

new_df_park['categories'] = new_df_park.apply(get_category_type, axis=1)

new_df_park.columns = [column.split('.')[-1] for column in new_df_park.columns]

new_df_park.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Queen's Park Subway Station,Metro Station,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",887,M5G 2P1,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",4b170e47f964a5208cc123e3
1,Grange Park,Park,NaN,NaN,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.65248755299856...",663,NaN,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, To...",4b54deadf964a520a3d027e3
2,TTC Streetcar #403 - Victoria park,Light Rail Station,NaN,NaN,43.650735,-79.386921,"[{'label': 'display', 'lat': 43.650735, 'lng':...",389,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",4b9177e4f964a520d2be33e3
3,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,btwn. College St. and Gerrard St.,43.659822,-79.385159,"[{'label': 'display', 'lat': 43.65982157793473...",712,M5G 2R3,CA,Toronto,ON,Canada,[763 Bay St. (btwn. College St. and Gerrard St...,4d3a1d23325ff04db2dd1845
4,St. Andrews Playground / Dog Park,Park,450 Adelaide Street West,Brant St & Adelaide St W,43.647388,-79.398752,"[{'label': 'display', 'lat': 43.64738759338116...",1372,NaN,CA,Toronto,ON,Canada,[450 Adelaide Street West (Brant St & Adelaide...,4bfc5019da7120a132df49fd


In [77]:
new_df_park= new_df_park.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
new_df_park

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
1,Grange Park,Park,NaN,43.652488,-79.392053,NaN,ON
2,TTC Streetcar #403 - Victoria park,Light Rail Station,NaN,43.650735,-79.386921,NaN,ON
3,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
4,St. Andrews Playground / Dog Park,Park,450 Adelaide Street West,43.647388,-79.398752,NaN,ON
5,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
6,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
7,Physiomed College Park,Medical Center,382 Yonge St,43.659240,-79.382860,M5B 1S8,ON
8,Berczy Park,Park,35 Wellington St. East,43.648048,-79.375172,NaN,ON
9,Moss Park Tennis Courts,Tennis Court,Moss Park,43.655337,-79.371923,NaN,ON


In [78]:
new_df_park = new_df_park.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
new_df_park

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
3,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
5,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
6,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
7,Physiomed College Park,Medical Center,382 Yonge St,43.659240,-79.382860,M5B 1S8,ON
10,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON
14,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
15,Regent Park Community Centre,Community Center,465 Dundas St E,43.659512,-79.363048,M5A 2B2,ON
17,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
18,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON


In [79]:
df_park = new_df_park[new_df_park.categories == 'Park']
df_park

,name,categories,address,lat,lng,postalCode,state
5,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
14,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
17,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
18,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
20,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
21,Barbara Hall Park,Park,519 Church St,43.666879,-79.381068,M4Y 2K9,ON
23,High Park,Park,1873 Bloor St. W,43.646479,-79.463425,M6R 2Z3,ON
35,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,M5V 3Z1,ON
38,Coronation Park,Park,711 Lake Shore Blvd. W,43.634553,-79.405403,M5V 3T7,ON
47,Osgoode Hall Park,Park,130 Queen St W,43.651105,-79.386269,M5H 2N5,ON


In [81]:
search_query = 'Restaurant'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=2VJSZMEZBVCII3HYWBCNGEDFTEI3JAVBB3BU0X1XRGN4LGM1&client_secret=0JYZ0FESUQ02YKG3WUKAAOIHTIMVKHM1TYXYBW1ML3PF4QGB&ll=43.6534817,-79.3839347&v=20200417&query=Restaurant&radius=10000&limit=50'

In [83]:
result_restaurant = requests.get(url).json()

venue_restaurant = result_restaurant['response']['venues']

Restaurant_df = json_normalize(venue_restaurant)
Restaurant_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4ad4c05ff964a52048f720e3,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1587139370,False,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,NaN
1,4ad4c05cf964a52006f620e3,Victoria's Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1587139370,False,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,498556908,NaN
2,4b266f05f964a520657b24e3,Goldstone Noodle Restaurant 金石,"[{'id': '4bf58dd8d48988d1d1941735', 'name': 'N...",v-1587139370,False,266 Spadina Ave,43.652278,-79.398039,"[{'label': 'display', 'lat': 43.6522783893466,...",1143,M5T 2E4,CA,Toronto,ON,Canada,"[266 Spadina Ave (at Willison Sq), Toronto ON ...",at Willison Sq,NaN,NaN
3,4ada5d5bf964a520e92121e3,The Hot House Restaurant & Bar,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1587139370,False,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",973,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,NaN
4,4ae29812f964a520288f21e3,Swatow Restaurant 汕頭小食家,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1587139370,False,309 Spadina Ave.,43.653866,-79.398334,"[{'label': 'display', 'lat': 43.65386562507761...",1160,M5T 2E6,CA,Toronto,ON,Canada,[309 Spadina Ave. (btwn Dundas St. W & D'Arcy ...,btwn Dundas St. W & D'Arcy St.,NaN,NaN


In [84]:
Restaurant_new_columns = ['name', 'categories'] + [col for col in Restaurant_df.columns if col.startswith('location.')]+ ['id']
new_df_restaurant = Restaurant_df.loc[:,Restaurant_new_columns]

def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

new_df_restaurant['categories'] = new_df_restaurant.apply(get_category_type, axis=1)

new_df_restaurant.columns = [column.split('.')[-1] for column in new_df_restaurant.columns]

new_df_restaurant.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,4ad4c05ff964a52048f720e3
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,NaN,4ad4c05cf964a52006f620e3
2,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,"[{'label': 'display', 'lat': 43.6522783893466,...",1143,M5T 2E4,CA,Toronto,ON,Canada,"[266 Spadina Ave (at Willison Sq), Toronto ON ...",at Willison Sq,NaN,4b266f05f964a520657b24e3
3,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",973,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,4ada5d5bf964a520e92121e3
4,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,"[{'label': 'display', 'lat': 43.65386562507761...",1160,M5T 2E6,CA,Toronto,ON,Canada,[309 Spadina Ave. (btwn Dundas St. W & D'Arcy ...,btwn Dundas St. W & D'Arcy St.,NaN,4ae29812f964a520288f21e3


In [85]:
new_df_restaurant= new_df_restaurant.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
new_df_restaurant

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
2,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON
3,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,M5E 1T3,ON
4,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,M5T 2E6,ON
5,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,M5T 2G3,ON
6,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,43.652783,-79.398174,NaN,ON
7,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,43.646463,-79.389644,M5V 1J5,ON
8,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,NaN,ON
9,360 Restaurant,Wine Bar,301 Front St W,43.642537,-79.387042,M5V 2T6,ON


In [86]:
df_Restaurant = new_df_restaurant.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
2,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON
3,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,M5E 1T3,ON
4,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,M5T 2E6,ON
5,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,M5T 2G3,ON
7,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,43.646463,-79.389644,M5V 1J5,ON
9,360 Restaurant,Wine Bar,301 Front St W,43.642537,-79.387042,M5V 2T6,ON
10,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,M5B,ON
11,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON


In [88]:
neighbourhood_df = pd.concat([df_hotel, df_park, df_Restaurant], ignore_index=True)
neighbourhood_df

,name,categories,address,lat,lng,postalCode,state,neighborhood
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON,NaN
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON,NaN
2,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON,NaN
3,Pantages Hotel & Spa,Hotel,200 Victoria St,43.654498,-79.379035,NaN,ON,NaN
4,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,43.658094,-79.382711,NaN,ON,NaN
5,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON,NaN
6,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON,NaN
7,Stathcona Hotel,Hotel,NaN,43.654947,-79.386359,NaN,ON,NaN
8,650 Hotel,Hotel,650 Bay Street,43.657046,-79.384411,NaN,ON,NaN
9,Be SixFifty Hotel,Hotel,650 Bay Street,43.657120,-79.384560,NaN,ON,NaN


In [89]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(neighbourhood_df['lat'], neighbourhood_df['lng'], 
                                           neighbourhood_df['name'], neighbourhood_df['categories'],\
                                               neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto